In [1]:
import pandas as pd
import numpy as np

<img src='img/Снимок.png'>

In [2]:
df = pd.read_csv('cs-training.csv').drop(columns='Unnamed: 0')
df.head(2)

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,0.766127,45,2,0.802982,9120.0,13,0,6,0,2.0
1,0,0.957151,40,0,0.121876,2600.0,4,0,0,0,1.0


In [3]:
df.describe()

,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
count,150000.000000,150000.000000,150000.000000,150000.000000,150000.000000,1.202690e+05,150000.000000,150000.000000,150000.000000,150000.000000,146076.000000
mean,0.066840,6.048438,52.295207,0.421033,353.005076,6.670221e+03,8.452760,0.265973,1.018240,0.240387,0.757222
std,0.249746,249.755371,14.771866,4.192781,2037.818523,1.438467e+04,5.145951,4.169304,1.129771,4.155179,1.115086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.029867,41.000000,0.000000,0.175074,3.400000e+03,5.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.154181,52.000000,0.000000,0.366508,5.400000e+03,8.000000,0.000000,1.000000,0.000000,0.000000
75%,0.000000,0.559046,63.000000,0.000000,0.868254,8.249000e+03,11.000000,0.000000,2.000000,0.000000,1.000000
max,1.000000,50708.000000,109.000000,98.000000,329664.000000,3.008750e+06,58.000000,98.000000,54.000000,98.000000,20.000000


## Исследование данных
### 1. RevolvingUtilizationOfUnsecuredLines

Отношение суммарного овердрафта по всем картам, к суммарному пределу по этим карта. На каждую карту не варьируется от 0 до 1. Косвенно может подсказать количество имеющихся карт.

In [4]:
df.RevolvingUtilizationOfUnsecuredLines.quantile(q=0.998)

2.761009013480203

99,8 % наблюдений показывают адекватные, вероятные в жизни значения овердрафтов. Значения выше скорее всего являются либо ошибкой ввода, либо каким то кодом. Проверим вид аномальных значений.

In [5]:
pd.options.display.max_rows=100

df.RevolvingUtilizationOfUnsecuredLines[df.RevolvingUtilizationOfUnsecuredLines>4]

293        2340.000000
697        2066.000000
1991       1143.000000
2331       6324.000000
4278       1982.000000
              ...     
148828       73.846154
149102     6109.000000
149160    22000.000000
149245      771.000000
149279    20514.000000
Name: RevolvingUtilizationOfUnsecuredLines, Length: 264, dtype: float64

Значения порядка десятков, сотен и тысяч можно сократить до единиц и десятых. 

In [6]:
# Статистика до преобразования
df.RevolvingUtilizationOfUnsecuredLines.describe()

count    150000.000000
mean          6.048438
std         249.755371
min           0.000000
25%           0.029867
50%           0.154181
75%           0.559046
max       50708.000000
Name: RevolvingUtilizationOfUnsecuredLines, dtype: float64

In [7]:
for n in range(4):
    df.RevolvingUtilizationOfUnsecuredLines = np.where(df.RevolvingUtilizationOfUnsecuredLines>(10000/10**n), 
                                                       df.RevolvingUtilizationOfUnsecuredLines/(100000/10**n), 
                                                       df.RevolvingUtilizationOfUnsecuredLines)

In [8]:
# Статистика после преобразования
df.RevolvingUtilizationOfUnsecuredLines.describe()

count    150000.000000
mean          0.322877
std           0.366610
min           0.000000
25%           0.029867
50%           0.153994
75%           0.555956
max           8.851852
Name: RevolvingUtilizationOfUnsecuredLines, dtype: float64

### 2. age

Возраст заемщика. Коррелирует с социальным статусом и благополучием до определенного момента (пенсионный возраст) после которого по состоянию здоровья и доходу выдача кредита становится рискованной.

In [9]:
pd.options.display.max_rows=10
df.age.value_counts()

49     3837
48     3806
50     3753
63     3719
47     3719
       ... 
101       3
109       2
107       1
105       1
0         1
Name: age, Length: 86, dtype: int64

Возраст 0 лет является скорее всего каким то пропуском первой значащей цифры (30,40,50). Аномальные возраста можно объединить к полуоткрытой группе старше определенного возраста.  
Попробуем объеденить различные значения возраста на возрастные категории, например, по рекомендации НБКИ:
* До 24 лет (для США 21-24)
* 25-34 года
* 35-44 года
* 45-59 лет
* 60 лет и старше

In [10]:
# Сначала промежуточно поделим на численные категории, потому что numpy.where 
# не умеет фильтровать переменные смешанного типа
df.age = np.where( (df.age == 0), np.median(df.age), df.age )
df.age = np.where( (df.age >= 21)&(df.age <= 24), 1, df.age )
df.age = np.where( (df.age >= 25)&(df.age <= 34), 2, df.age )
df.age = np.where( (df.age >= 35)&(df.age <= 44), 3, df.age )
df.age = np.where( (df.age >= 45)&(df.age <= 59), 4, df.age )
df.age = np.where( (df.age >= 60), 5, df.age )

df.age = df.age.apply(int)
df.age.value_counts()

4    53880
5    48318
3    28563
2    17165
1     2074
Name: age, dtype: int64

In [11]:
# Теперь одиночные числовые метки можно преобразовать в строковые категории. 
# Это нужно для последующего дамми-кодирования.
df.age = np.where( df.age == 1, "До 24 лет", df.age )
df.age = np.where( df.age == '2', "25-34 года", df.age )
df.age = np.where( df.age == '3', "35-44 года", df.age )
df.age = np.where( df.age == '4', "45-59 лет", df.age )
df.age = np.where( df.age == '5', "Старше 60 лет", df.age )
df.age.value_counts()

45-59 лет        53880
Старше 60 лет    48318
35-44 года       28563
25-34 года       17165
До 24 лет         2074
Name: age, dtype: int64

### 3. NumberOfTime30-59DaysPastDueNotWorse / NumberOfTime60-89DaysPastDueNotWorse / NumberOfTimes90DaysLate
Количество просрочек кредита на срок 30-59 дней, 60-89 дней и свыше 90 дней. 

In [12]:
df[['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse',
    'NumberOfTimes90DaysLate']].describe()

,NumberOfTime30-59DaysPastDueNotWorse,NumberOfTime60-89DaysPastDueNotWorse,NumberOfTimes90DaysLate
count,150000.000000,150000.000000,150000.000000
mean,0.421033,0.240387,0.265973
std,4.192781,4.155179,4.169304
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000
max,98.000000,98.000000,98.000000


In [13]:
for col in ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']:
    print(df[col].value_counts())

0     126018
1      16033
2       4598
3       1754
4        747
       ...  
96         5
10         4
12         2
13         1
11         1
Name: NumberOfTime30-59DaysPastDueNotWorse, Length: 16, dtype: int64
0     142396
1       5731
2       1118
3        318
98       264
       ...  
7          9
96         5
8          2
11         1
9          1
Name: NumberOfTime60-89DaysPastDueNotWorse, Length: 13, dtype: int64
0     141662
1       5243
2       1555
3        667
4        291
       ...  
13         4
12         2
14         2
15         2
17         1
Name: NumberOfTimes90DaysLate, Length: 19, dtype: int64


Подозрительные значения 96 и 98 повторяются часто и вероятно что то значат, выделим их в отдельную категорию.  
Также выделим категории:
* Нет просрочек
* Одна просрочка 
* Две и более просрочек

In [14]:
for col in ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']:
    
    df[col] = np.where( (df[col] == 96)|(df[col] == 98), 9999, df[col] )
    df[col] = np.where( (df[col] == 1), 1, df[col] )
    df[col] = np.where( (df[col] >= 2)&(df[col] <= 95), 2, df[col] )

In [15]:
for col in ['NumberOfTime30-59DaysPastDueNotWorse','NumberOfTime60-89DaysPastDueNotWorse','NumberOfTimes90DaysLate']:
    
    df[col] = np.where( (df[col] == 9999),'Аномалия' , df[col] )
    df[col] = np.where( (df[col] == '0'), 'Нет просрочк', df[col] )
    df[col] = np.where( (df[col] == '1'), 'Одна просрочка', df[col] )
    df[col] = np.where( (df[col] == '2'), 'Более двух просрочек', df[col] )
df['NumberOfTime30-59DaysPastDueNotWorse'].value_counts()

Нет просрочк            126018
Одна просрочка           16033
Более двух просрочек      7680
Аномалия                   269
Name: NumberOfTime30-59DaysPastDueNotWorse, dtype: int64

### 4 DebtRatio
Отношение ежемесячного платежа к ежемесячному доходу.  Значения свыше единицы особенно на много порядков, означают, скорее всего ошибку ввода. 

In [16]:
df.DebtRatio.describe()

count    150000.000000
mean        353.005076
std        2037.818523
min           0.000000
25%           0.175074
50%           0.366508
75%           0.868254
max      329664.000000
Name: DebtRatio, dtype: float64

In [17]:
pd.set_option("display.max_rows", 101)
df.DebtRatio[df.DebtRatio<100].max() 

99.0

In [18]:
# Приведем к десятичному значению.
for n in range(6):
    df.DebtRatio = np.where(df.DebtRatio>(100000/10**n), df.DebtRatio/(1000000/10**n), df.DebtRatio)

df.DebtRatio.describe()

count    150000.000000
mean          0.310706
std           0.227837
min           0.000000
25%           0.138801
50%           0.269285
75%           0.440000
max           1.000000
Name: DebtRatio, dtype: float64

### 5. MonthlyIncome
Ежемесячный доход неограничен сверху, но ограничен снизу согласно МРОТ. Для США (по данным 2016) с учетом чаевых МРОТ составляет порядка 340 долларов в месяц.

In [19]:
pd.set_option('float_format',lambda x: "%.2f" % x)
df.MonthlyIncome.describe()

count    120269.00
mean       6670.22
std       14384.67
min           0.00
25%        3400.00
50%        5400.00
75%        8249.00
max     3008750.00
Name: MonthlyIncome, dtype: float64

In [20]:
df.MonthlyIncome = np.where(df.MonthlyIncome < 1200, 1200, df.MonthlyIncome)
df.MonthlyIncome.describe()

count    120269.00
mean       6703.42
std       14370.85
min        1200.00
25%        3400.00
50%        5400.00
75%        8249.00
max     3008750.00
Name: MonthlyIncome, dtype: float64

### 6. NumberOfOpenCreditLinesAndLoans
Число открытых кредитов. Теоретически неограниченно, если они небольшие (книга в кредит по 1$ ежемесячный платеж на год)

In [21]:
df.NumberOfOpenCreditLinesAndLoans.describe()

count   150000.00
mean         8.45
std          5.15
min          0.00
25%          5.00
50%          8.00
75%         11.00
max         58.00
Name: NumberOfOpenCreditLinesAndLoans, dtype: float64

### 7. NumberRealEstateLoansOrLines
Количество ипотек. 

In [22]:
df.NumberRealEstateLoansOrLines.describe()

count   150000.00
mean         1.02
std          1.13
min          0.00
25%          0.00
50%          1.00
75%          2.00
max         54.00
Name: NumberRealEstateLoansOrLines, dtype: float64

In [23]:
df.NumberRealEstateLoansOrLines.value_counts()

0     56188
1     52338
2     31522
3      6300
4      2170
5       689
6       320
7       171
8        93
9        78
10       37
11       23
12       18
13       15
14        7
15        7
16        4
17        4
25        3
18        2
19        2
20        2
23        2
32        1
21        1
26        1
29        1
54        1
Name: NumberRealEstateLoansOrLines, dtype: int64

Предполагаемые категории:
* Нет ипотеки
* Ипотека за свое жилье
* Ипотека за два объекта
* Ипотека за множество объектов

In [24]:
for col in ['NumberRealEstateLoansOrLines']:
    df[col] = np.where( (df[col] > 2), 9999, df[col] )
    df[col] = np.where( (df[col] == 0), 'Нет ипотеки', df[col] )
    df[col] = np.where( (df[col] == '1'), 'Ипотека за свое жилье', df[col] )
    df[col] = np.where( (df[col] == '2'), 'Ипотека за два объекта', df[col] )
    df[col] = np.where( (df[col] == '9999'), 'Ипотека за множество объектов', df[col] )

    
df.NumberRealEstateLoansOrLines.value_counts()

Нет ипотеки                      56188
Ипотека за свое жилье            52338
Ипотека за два объекта           31522
Ипотека за множество объектов     9952
Name: NumberRealEstateLoansOrLines, dtype: int64

### 8. NumberOfDependents
Число иждивенцев теоретически неограничено. Десятки иждивенцев - характерный портрет иммигрантов с Ближнего Востока, живущих в традиционной семье. 

In [25]:
df.NumberOfDependents.value_counts()

0.00     86902
1.00     26316
2.00     19522
3.00      9483
4.00      2862
5.00       746
6.00       158
7.00        51
8.00        24
9.00         5
10.00        5
13.00        1
20.00        1
Name: NumberOfDependents, dtype: int64

In [26]:
df.NumberOfDependents.value_counts()

0.00     86902
1.00     26316
2.00     19522
3.00      9483
4.00      2862
5.00       746
6.00       158
7.00        51
8.00        24
9.00         5
10.00        5
13.00        1
20.00        1
Name: NumberOfDependents, dtype: int64

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 11 columns):
SeriousDlqin2yrs                        150000 non-null int64
RevolvingUtilizationOfUnsecuredLines    150000 non-null float64
age                                     150000 non-null object
NumberOfTime30-59DaysPastDueNotWorse    150000 non-null object
DebtRatio                               150000 non-null float64
MonthlyIncome                           120269 non-null float64
NumberOfOpenCreditLinesAndLoans         150000 non-null int64
NumberOfTimes90DaysLate                 150000 non-null object
NumberRealEstateLoansOrLines            150000 non-null object
NumberOfTime60-89DaysPastDueNotWorse    150000 non-null object
NumberOfDependents                      146076 non-null float64
dtypes: float64(4), int64(2), object(5)
memory usage: 12.6+ MB


## Подготовка конвейера

In [28]:
# Списки столбцов
cat_columns = df.dtypes[df.dtypes == 'object'].index
num_columns = df.drop(columns='SeriousDlqin2yrs').dtypes[df.dtypes != 'object'].index

In [29]:
from sklearn.preprocessing import OneHotEncoder, PowerTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [30]:
# Конвейер категориальных переменных 
cat_pipe = Pipeline([('imp', SimpleImputer(strategy='most_frequent')),('ohe', OneHotEncoder(sparse=False))])

# Конвейер численных переменных
num_pipe = Pipeline([('imp', SimpleImputer(strategy='median')), ('power-trafo', PowerTransformer())])

In [31]:
# Класс ColumnTransformer
trafo = [
    ('num', num_pipe, num_columns),
    ('cat', cat_pipe, cat_columns)    
]

transformer = ColumnTransformer(transformers=trafo)

In [32]:
ml_pipe = Pipeline([('tr', transformer),
                     ('lr', LogisticRegression(solver='liblinear',max_iter=200))])

In [33]:
roc =  round(cross_val_score(ml_pipe, 
                df.drop(columns='SeriousDlqin2yrs'), 
                df['SeriousDlqin2yrs'], 
                n_jobs=8, 
                cv=12, 
                scoring='roc_auc').mean(), 3)

In [34]:
roc

0.856